#### model = Concat signatures with pose -> Dense -> softmax with 100 classes. 
#### run model for each frame in a seq, and then take majority vote

In [1]:
from data import *

In [2]:
pose_paths, keypoints, scores = read_pose('data/pose.pkl')
signatures_paths, signatures = read_signatures('data/signatures.pkl')

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

IMAGE_DIM = 64
NUM_CLASSES = 100
learning_rate = 1e-4
epochs = 500
INPUT_DIM = 2048 # signatures[0].shape[0]


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(IMAGE_DIM * IMAGE_DIM, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, NUM_CLASSES)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)
    
net = Net()
print(net)

optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
loss_fn = nn.NLLLoss()

Net(
  (fc1): Linear(in_features=4096, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=100, bias=True)
)


In [4]:
# from tqdm import tqdm
# for t in tqdm(range(epochs)):
#     y_pred = net(x)
#     loss = loss_fn(y_pred, y)
#     if t % 100 == 99:
#         print(t, loss.item())
#     model.zero_grad()
#     loss.backward()
#     optimizer.step()

In [5]:
from torch.utils.data import Dataset, DataLoader
import torch
import os
DATA_DIR = 'data/'
IMAGES_FILE = 'images.tar'
POSE_FILE = 'pose.pkl'
SIGNATURES_FILE = 'signatures.pkl'

IMAGES_FILE_TEST = 'images_test.tar'
POSE_FILE_TEST = 'pose_test.pkl'
SIGNATURES_FILE_TEST = 'signatures_test.pkl'

In [6]:
def get_pose_dists(pose):
    return torch.Tensor([x for x in np.array([np.sqrt(sum((kp_1 - kp_2) ** 2, 0)) for kp_1 in pose for kp_2 in pose]) if x != 0])

In [7]:
class OrcamDataset(Dataset):
    def __init__(self, data_dir, train=True, filter_signatures=True):
        self.data_dir = data_dir
        im_file = os.path.join(data_dir, IMAGES_FILE if train else IMAGES_FILE_TEST)
        pose_file = os.path.join(data_dir, POSE_FILE if train else POSE_FILE_TEST)
        sig_file = os.path.join(data_dir, SIGNATURES_FILE if train else SIGNATURES_FILE_TEST)
        self.pose_paths, self.keypoints, self.scores = read_pose(pose_file)
        self.signatures_paths, self.signatures = read_signatures(sig_file)
        self.signatures = torch.Tensor(self.signatures)
        self.images = Images(im_file)
        self.actual_paths = list(set(self.images.paths) & set(self.signatures_paths))
    def __len__(self):
        return len(self.actual_paths)

    @staticmethod
    def get_label(path):
        slash = path.find('/')
        underscore = path.find('_')
        num_str = path[underscore + 1:slash]
        return int(num_str)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        path = self.actual_paths[idx]
        sign_idx = self.signatures_paths.index(path)
        signature = self.signatures[sign_idx]
        pose_idx = self.pose_paths.index(path)
        pose = self.keypoints[pose_idx]
        image = self.images[path]
        image = torch.Tensor(image.copy())
        label = self.get_label(path)
        pose_dist = get_pose_dists(pose)
        sample = {'image': image, "signature": signature, "pose": pose_dist, "label": torch.Tensor([label])}
        return sample


In [11]:
import matplotlib.pyplot as plt
dataset = OrcamDataset(DATA_DIR, train=True)
dataload = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=1)

In [11]:
n = 10
for i, batch in enumerate(dataload):
    if i == n:
        break
    # break
    print(batch['image'].size())
    print(batch['signature'].size())
    print(batch['pose'].size())
#     ax = plt.subplot(1, n, i+1)
#     plt.tight_layout()
#     ax.set_title('#{}'.format(i))
#     ax.axis('off')
#     plt.imshow(image)
    
    

torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
torch.Size([16, 64, 64, 3])
torch.Size([16, 2048])
torch.Size([16, 272])
